# TACCSSHClient Class

The TACCSSHClient class is used to establish connections with TACC systems, using a user's login and 2FA code.

The class then provides methods for (1)basic file operations, and for (2) execution of arbitrary commands.
We will go over these two main areas of functionality in the tutorial

## Initialization

To initialize a `TACCSSHClient` class object, you will to pass in the system you want to connect to. Currently supported systems include:

- stampede2 - [Stampede2](https://portal.tacc.utexas.edu/user-guides/stampede)
- ls6 - [Lonestar6](https://portal.tacc.utexas.edu/user-guides/lonestar6)
- frontera - [Frontera](https://frontera-portal.tacc.utexas.edu/user-guide/)
- maverick2 - [Maverick2](https://portal.tacc.utexas.edu/user-guides/maverick2)


In [1]:
from taccjm.TACCSSHClient import TACCSSHClient
tsc = TACCSSHClient('ls6', 'clos21')

Password: ········
TACC Token Code: 247738


## Command Operations

### `execute_command()`

The TACCSSHClient offers the `execute_command()` function to execute arbitrary commands as if one were in an ssh session interactively. 
The commands are executed on a separately opened channel, spawned from the original paramiko transport object used to establish the main ssh connection when the class was initialized.

In [2]:
print(tsc.execute_command('showq -u clos21')['stdout'])

Using system default configuration file 

SUMMARY OF JOBS FOR USER: <clos21>

ACTIVE JOBS--------------------
JOBID     JOBNAME    USERNAME      STATE   NODES REMAINING STARTTIME

WAITING JOBS------------------------
JOBID     JOBNAME    USERNAME      STATE   NODES WCLIMIT   QUEUETIME

Total Jobs: 0     Active Jobs: 0     Idle Jobs: 0     Blocked Jobs: 0   



### Command Errors

If the command fails, an error will be thrown:

In [3]:
tsc.execute_command('showr')['stderr']

{"asctime": "2023-02-07 17:36:29,795", "name": "taccjm.TACCSSHClient", "levelname": "ERROR", "message": "Non-zero return code.\nclos21@ls6.tacc.utexas.edu$ showr\nrc     : 127\nstdout : \nstderr : bash: showr: command not found\n"}


SSHCommandError: Non-zero return code.
clos21@ls6.tacc.utexas.edu$ showr
rc     : 127
stdout : 
stderr : bash: showr: command not found


As we can see, we get a good error clearly indicating:

- system on which command was run (Lonestar6)
- User running command (clos21)
- Command being run, after the "$" sign, (showr)
- The return code (127), stdout, and stderr

So we can clearly see here that the command failed because `showr` is not a valid command. We probably meant `showq`

We can also supress the error by setting `error=False`.
This allows use to check the error code/error message and handle it as necessary.

In [4]:
tsc.execute_command('showr', error=False)

{'id': 4,
 'cmd': 'showr',
 'ts': datetime.datetime(2023, 2, 7, 17, 36, 31, 439860),
 'status': 'FAILED',
 'stdout': '',
 'stderr': 'bash: showr: command not found\n',
 'history': [{'ts': datetime.datetime(2023, 2, 7, 17, 36, 30, 288107),
   'status': 'STARTED'}],
 'rt': 1,
 'rc': 127}

### `process_command()` - Asynchronous commands

Commands can also be run in the background, for example if performing a long copy or monitoring some job. 
This is done by set the `wait=False` parameter, and then using the `process_command()` method to check on the status of the command using the command's ID.

In [5]:
cmnd = tsc.execute_command('echo START; sleep 10; echo START', wait=False)
cmnd

{'id': 5,
 'cmd': 'echo START; sleep 10; echo START',
 'ts': datetime.datetime(2023, 2, 7, 17, 36, 31, 571251),
 'status': 'STARTED',
 'stdout': '',
 'stderr': '',
 'history': [],
 'channel': <paramiko.Channel 4 (open) window=2097152 -> <paramiko.Transport at 0x3d257790 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>,
 'rt': None}

In [6]:
cmnd = tsc.process_command(cmnd['id'])
cmnd

{'id': 5,
 'cmd': 'echo START; sleep 10; echo START',
 'ts': datetime.datetime(2023, 2, 7, 17, 36, 31, 712024),
 'status': 'RUNNING',
 'stdout': '',
 'stderr': '',
 'history': [{'ts': datetime.datetime(2023, 2, 7, 17, 36, 31, 571251),
   'status': 'STARTED'}],
 'channel': <paramiko.Channel 4 (open) window=2097152 -> <paramiko.Transport at 0x3d257790 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>,
 'rt': None}

Note the command's history of status's is stored.
This history is compiled as the process is polled, with duplicate status's removed.
Finally note how the `rt` field gives us a rough runtime for the command.
Once again if the process was run in the background, this runtime is according to how the process was polled, not necessarily how long the command took to run.

### Buffering stdout from commands

You can pass an `nbytes: int=None` parameter to the `process_command()` method to read the next `nbytes` available in the stdout buffer of an actively running command:

In [7]:
import time 

cmnd = tsc.execute_command('echo START; sleep 7; echo 1; sleep 6; echo 2', wait=False)

t = 0
while cmnd['status'] not in ['COMPLETE', 'FAILED']:
    cmnd = tsc.process_command(cmnd['id'], wait=False, nbytes=5)
    print(f"stdout at time {t}: {cmnd['stdout']}")
    t += 5
    time.sleep(5)

stdout at time 0: START
stdout at time 5: START

stdout at time 10: START
1

stdout at time 15: START
1
2



Note how the output is appended to the 'stdout' field of the command config as it becomes available, and only `nbytes` are read on any given call.

## File Operations

The TACCSHClient class uses its underlying ssh connection to open sftp connections and provide programmatic access to files on TACC systems.
The following operations are offered to interact with files on TACC systems using the TACCSSHClient Class:

- list_files - Get file and directory info (file stats)
- upload - Upload a file/directory to TACC system via sftp.
- downlaod - Download a file/directory from TACC system via sftp.
- read - Read file contents (text or json) directly from TACC system via sftp.
- write - Write data (string or dictioanry) directly from TACC system via sftp.

Note:

    1. All remote paths (paths on TACC systems) must use unix path separators '/'.
    2. File operations currently aren't "safe." 
       That is file uploads/downloads and reads/writes will overwrite contents on remote/local systems.
    3. If absolute paths aren't specified, paths are relative to a user's $HOME diretory on TACC systems, and relative to the cwd for the local paths.

### `list_files()`

The `list_files()` method provides basic file stat info, much like an `ls` command would in a shell session.

In [9]:
files = tsc.list_files('')
len(files)

15

In [10]:
files[0]

{'filename': '11',
 'st_atime': 1660905150,
 'st_gid': 800588,
 'st_mode': 16832,
 'st_mtime': 1660905152,
 'st_size': 4,
 'st_uid': 856065,
 'ls_str': b'drwx------   1 856065   800588          4 19 Aug 05:32 11'}

In [11]:
print('\n'.join([x['ls_str'].decode('utf-8') for x in files if not x['filename'].startswith('.')]))

drwx------   1 856065   800588          4 19 Aug 05:32 11
drwx------   1 856065   800588          4 03 Jun 2022  taccjm
drwx------   1 856065   800588          4 08 Sep 22:48 test-taccjm
drwx------   1 856065   800588          1 02 Feb 12:00 ch-sim-tests
drwx------   1 856065   800588          4 09 Sep 12:33 l2
drwx------   1 856065   800588          4 14 Dec 2021  ls6
drwx------   1 856065   800588          9 12 May 2022  temp
drwx------   1 856065   800588          7 02 Dec 2020  si-base
drwx------   1 856065   800588          4 31 Jan 16:28 ch-sim-ls6
drwx------   1 856065   800588          4 11 Jan 2022  test
drwx------   1 856065   800588          2 30 Jan 15:54 ch-sim
drwx------   1 856065   800588          1 09 Sep 18:29 ios-output
drwx------   1 856065   800588          4 19 Aug 05:48 l1
drwx------   1 856065   800588          4 30 Sep 20:52 test_ls6
drwx------   1 856065   800588          4 25 Aug 11:21 ls1


In [12]:
print(tsc.execute_command('ls -lt $SCRATCH')['stdout'])

total 8
drwx------ 2 clos21 G-800588 1 Feb  2 12:00 ch-sim-tests
drwx------ 6 clos21 G-800588 4 Jan 31 16:28 ch-sim-ls6
drwx------ 4 clos21 G-800588 2 Jan 30 15:54 ch-sim
drwx------ 6 clos21 G-800588 4 Sep 30 20:52 test_ls6
drwx------ 3 clos21 G-800588 1 Sep  9 18:29 ios-output
drwx------ 6 clos21 G-800588 4 Sep  9 12:33 l2
drwx------ 6 clos21 G-800588 4 Sep  8 22:48 test-taccjm
drwx------ 6 clos21 G-800588 4 Aug 25 11:21 ls1
drwx------ 6 clos21 G-800588 4 Aug 19 05:48 l1
drwx------ 6 clos21 G-800588 4 Aug 19 05:32 11
drwx------ 6 clos21 G-800588 4 Jun  3  2022 taccjm
drwx------ 2 clos21 G-800588 9 May 12  2022 temp
drwx------ 6 clos21 G-800588 4 Jan 11  2022 test
drwx------ 6 clos21 G-800588 4 Dec 14  2021 ls6
drwx------ 3 clos21 G-800588 7 Dec  2  2020 si-base



### `upload()`

To upload a local directory or file to TACC, use the upload method.

In [38]:
!rm -rf test; mkdir test; echo test > test/test.txt

In [39]:
tsc.upload('test/test.txt', 'test.txt')

In [40]:
tsc.list_files('test.txt')

[{'filename': 'test.txt',
  'st_atime': 1675808371,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675808371,
  'st_size': 5,
  'st_uid': 856065,
  'ls_str': b'-rw-------   1 856065   800588          5 07 Feb 16:19 ?'}]

In [43]:
tsc.upload('test', 'test')

In [44]:
tsc.list_files('test')

[{'filename': 'test.txt',
  'st_atime': 1675808477,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675808368,
  'st_size': 5,
  'st_uid': 856065,
  'ls_str': b'-rw-------   1 856065   800588          5 07 Feb 16:19 test.txt'}]

### `download()`

In [45]:
tsc.download('test.txt', 'foo.txt')

In [47]:
!cat foo.txt

test


In [51]:
tsc.download('test', 'foo')

In [52]:
!ls foo/test

test.txt


### `write()`

In [54]:
tsc.write('hello world!\n', 'test2.txt')

In [55]:
tsc.list_files('test2.txt')

[{'filename': 'test2.txt',
  'st_atime': 1675808673,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675808673,
  'st_size': 13,
  'st_uid': 856065,
  'ls_str': b'-rw-------   1 856065   800588         13 07 Feb 16:24 ?'}]

### `read()`

In [56]:
msg = tsc.read('test2.txt')
print(msg)

hello world!



# TACC SSH API - tacc_ssh_api.py

The problem with managing individual TACCSSHClient classes is that they may time out after a period of inactivity, requiring you to login again.
Instead of using the TACCSSHClient class directly, you can use the `tacc_ssh_api` to spawn a server that manages TACCSSHClient classes for you.
These connections are kept alive (for as long as the server isn't restarted), and they can be interacted via methods in `tacc_ssh_api`.
The main methods provided to interact with the SSH server are the following:

- `list()` - List active ssh session
- `get(connection_id: int)` - Get a session an active session
- `init(...)` - Initialize an ssh session (by initializing a new TACCSSHClient object)
- `exec(connection_id:int, ...)` - Run a command using `execute_command()` on a given session.
- `process(connection_id:int, cmnd_id:int ...)` - Process a command run on a given session.
- `list_files(connection_id:int, path:str ...)` - List files.
- `read(connection_id:int, path:str ...)` - Read a file directly via sftp.
- `write(connection_id:int, data, path:str)` - Write a file directly via sftp.
- `upload(connection_id:int, local_path:str, remote_path:str)` - Upload files/folders
- `download(connection_id:int, data, remote_path:str, local_path:Str)` - Download files/folders

## Initializing a sessions

In [1]:
from taccjm import tacc_ssh_api as tsa

In [2]:
tsa.find_server(start=True, kill=True)

{'server': <Popen: returncode: None args: ['python', '/home/jovyan/work/repos/taccjm/sr...>,
 'hb': <Popen: returncode: None args: ['python', '/home/jovyan/work/repos/taccjm/sr...>}

In [3]:
tsa.init('test', 'ls6', 'clos21', 'Leo#21Wanda')

mfa:  138528


{'id': 'test',
 'sys': 'ls6.tacc.utexas.edu',
 'user': 'clos21',
 'start': '2023-02-07T22:39:48.288785',
 'last_ts': '2023-02-07T22:39:48.288787',
 'loglevel': '20',
 'logfile': '/home/jovyan/.taccjm/test_ls6_clos21_log.json'}

In [4]:
tsa.get('test')

{'id': 'test',
 'sys': 'ls6.tacc.utexas.edu',
 'user': 'clos21',
 'start': '2023-02-07T22:35:07.435852',
 'last_ts': '2023-02-07T22:35:07.435853',
 'loglevel': '20',
 'logfile': '/home/jovyan/.taccjm/test_ls6_clos21_log.json'}

## Command Operations

### `exec()` - General Command Execution

...

In [5]:
ls_cmnd = tsa.exec('test', 'ls $SCRATCH', wait=False)

### `process()` - Polling commands

Pass a command ID to poll a specific command. 
Set `wait=False`, unless you want to wait until the command finishes.
You can set `nbytes=int` to get the latest nbytes from the stdout buff .

In [13]:
ls_cmnd = tsa.exec('test', 'sleep 10; ls $SCRATCH', wait=False)

In [14]:
tsa.process('test', ls_cmnd['id'], wait=False)

{'id': 10,
 'cmd': 'sleep 10; ls $SCRATCH',
 'ts': '2023-02-07T18:22:49.523330',
 'status': 'RUNNING',
 'stdout': '',
 'stderr': '',
 'history': [{'ts': '2023-02-07T18:22:48.983625', 'status': 'STARTED'}],
 'rt': None}

### Processing all Active commands


In [5]:
cmnd_1 = tsa.exec('test', 'sleep 10; ls $SCRATCH', wait=False)
cmnd_2 = tsa.exec('test', 'sleep 20; ls $SCRATCH', wait=False)
cmnd_3 = tsa.exec('test', 'sleep 30; ls $SCRATCH', wait=False)

In [14]:
tsa.process('test')

[{'id': 3,
  'cmd': 'sleep 20; ls $SCRATCH',
  'ts': '2023-02-07T18:28:00.670353',
  'status': 'RUNNING',
  'stdout': '',
  'stderr': '',
  'history': [{'ts': '2023-02-07T18:27:43.070502', 'status': 'STARTED'}],
  'rt': None},
 {'id': 4,
  'cmd': 'sleep 30; ls $SCRATCH',
  'ts': '2023-02-07T18:28:00.670360',
  'status': 'RUNNING',
  'stdout': '',
  'stderr': '',
  'history': [{'ts': '2023-02-07T18:27:43.146205', 'status': 'STARTED'}],
  'rt': None}]

## File Operations

### `list_files()`

The `list_files()` method provides basic file stat info, much like an `ls` command would in a shell session.

In [21]:
files = tsa.list_files('test', 'test')
files

[{'filename': 'trash',
  'st_atime': 1641927888,
  'st_gid': 800588,
  'st_mode': 16832,
  'st_mtime': 1662657040,
  'st_size': 4,
  'st_uid': 856065,
  'ls_str': 'drwx------   1 856065   800588          4 08 Sep 12:10 trash'},
 {'filename': 'scripts',
  'st_atime': 1641927887,
  'st_gid': 800588,
  'st_mode': 16832,
  'st_mtime': 1661545891,
  'st_size': 1,
  'st_uid': 856065,
  'ls_str': 'drwx------   1 856065   800588          1 26 Aug 15:31 scripts'},
 {'filename': 'jobs',
  'st_atime': 1641927886,
  'st_gid': 800588,
  'st_mode': 16832,
  'st_mtime': 1641927886,
  'st_size': 0,
  'st_uid': 856065,
  'ls_str': 'drwx------   1 856065   800588          0 11 Jan 2022  jobs'},
 {'filename': 'apps',
  'st_atime': 1641927887,
  'st_gid': 800588,
  'st_mode': 16832,
  'st_mtime': 1661789712,
  'st_size': 3,
  'st_uid': 856065,
  'ls_str': 'drwx------   1 856065   800588          3 29 Aug 11:15 apps'}]

In [6]:
print('\n'.join([x['ls_str'] for x in files if not x['filename'].startswith('.')]))

drwx------   1 856065   800588          4 19 Aug 05:32 11
drwx------   1 856065   800588          4 03 Jun 2022  taccjm
drwx------   1 856065   800588          4 08 Sep 22:48 test-taccjm
drwx------   1 856065   800588          1 02 Feb 12:00 ch-sim-tests
drwx------   1 856065   800588          4 09 Sep 12:33 l2
drwx------   1 856065   800588          4 14 Dec 2021  ls6
drwx------   1 856065   800588          9 12 May 2022  temp
drwx------   1 856065   800588          7 02 Dec 2020  si-base
drwx------   1 856065   800588          4 31 Jan 16:28 ch-sim-ls6
drwx------   1 856065   800588          4 11 Jan 2022  test
drwx------   1 856065   800588          2 30 Jan 15:54 ch-sim
drwx------   1 856065   800588          1 09 Sep 18:29 ios-output
drwx------   1 856065   800588          4 19 Aug 05:48 l1
drwx------   1 856065   800588          4 30 Sep 20:52 test_ls6
drwx------   1 856065   800588          4 25 Aug 11:21 ls1


In [10]:
print(tsa.exec('test', 'ls -lt $SCRATCH')['stdout'])

total 8
drwx------ 2 clos21 G-800588 1 Feb  2 12:00 ch-sim-tests
drwx------ 6 clos21 G-800588 4 Jan 31 16:28 ch-sim-ls6
drwx------ 4 clos21 G-800588 2 Jan 30 15:54 ch-sim
drwx------ 6 clos21 G-800588 4 Sep 30 20:52 test_ls6
drwx------ 3 clos21 G-800588 1 Sep  9 18:29 ios-output
drwx------ 6 clos21 G-800588 4 Sep  9 12:33 l2
drwx------ 6 clos21 G-800588 4 Sep  8 22:48 test-taccjm
drwx------ 6 clos21 G-800588 4 Aug 25 11:21 ls1
drwx------ 6 clos21 G-800588 4 Aug 19 05:48 l1
drwx------ 6 clos21 G-800588 4 Aug 19 05:32 11
drwx------ 6 clos21 G-800588 4 Jun  3  2022 taccjm
drwx------ 2 clos21 G-800588 9 May 12  2022 temp
drwx------ 6 clos21 G-800588 4 Jan 11  2022 test
drwx------ 6 clos21 G-800588 4 Dec 14  2021 ls6
drwx------ 3 clos21 G-800588 7 Dec  2  2020 si-base



### `write()` and `read()`

Use to write directly to a file on TACC systems.

#### Text Data

In [4]:
tsa.write('test', 'hello world!', 'hello.txt')

In [5]:
tsa.read('test', 'hello.txt')

{'path': 'hello.txt', 'data_type': 'text', 'data': 'hello world!'}

#### json Data

In [7]:
tsa.write('test', {'msg':'hello world!', 'id': 12312}, 'hello.json')

In [8]:
tsa.read('test', 'hello.json')

{'path': 'hello.json',
 'data_type': 'json',
 'data': {'msg': 'hello world!', 'id': 12312}}

### `upload()` and `download()`

To upload a local directory or file to TACC, use the upload method.

In [6]:
!rm -rf test-upload; mkdir test-upload; echo test > test-upload/test.txt

#### Files

In [5]:
tsa.upload('test', 'test-upload/test.txt', 'upload_test.txt')

In [6]:
tsa.list_files('test', 'upload_test.txt')

[{'filename': '/scratch/06307/clos21/upload_test.txt',
  'st_atime': 1675830682,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675830682,
  'st_size': 5,
  'st_uid': 856065,
  'ls_str': '-rw-------   1 856065   800588          5 07 Feb 22:31 ?'}]

In [7]:
tsa.download('test', 'upload_test.txt', 'test-upload/download_test.txt')

'test-upload/download_test.txt'

In [8]:
!cat test-upload/download_test.txt

test


#### Directories

In [5]:
!echo 'another test' > test-upload/test2.txt

In [4]:
tsa.upload('test', 'test-upload', 'test-upload')

In [5]:
tsa.list_files('test', 'test-upload')

[{'filename': 'download_test.txt',
  'st_atime': 1675831198,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675830687,
  'st_size': 5,
  'st_uid': 856065,
  'ls_str': '-rw-------   1 856065   800588          5 07 Feb 22:31 download_test.txt'},
 {'filename': 'test2.txt',
  'st_atime': 1675831199,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675830915,
  'st_size': 13,
  'st_uid': 856065,
  'ls_str': '-rw-------   1 856065   800588         13 07 Feb 22:35 test2.txt'},
 {'filename': 'test.txt',
  'st_atime': 1675831199,
  'st_gid': 800588,
  'st_mode': 33152,
  'st_mtime': 1675830340,
  'st_size': 5,
  'st_uid': 856065,
  'ls_str': '-rw-------   1 856065   800588          5 07 Feb 22:25 test.txt'}]

In [6]:
tsa.download('test', 'test-upload', 'test-download')

'test-download'

In [8]:
!ls -R test-download

test-download:
test-upload

test-download/test-upload:
download_test.txt  test2.txt  test.txt
